## Ubicaciones y sus respectivos locales

In [3]:
import statistics as stats
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import feedparser

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC   # acciones encadenadas
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options



import tqdm
from bs4 import BeautifulSoup
import time
import re
import json

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')

from pymongo import MongoClient

In [4]:
options=Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-allow-origins=*")
options.add_argument("user-data-dir=cookies")
#options.add_argument('--headless')                 #Habilitar si no queremos ver la ventana
options.add_experimental_option("detach", True)    #Esta opción corrige el error de cierre repentino
options.add_argument('--start-minimized')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')

Tras importar las librerías necesarias y ejecutar las opciones, doy paso a retirar las cookies del driver que voy a utilizar. 
- Al habilitar la opcion que mantiene las cookies eliminadas, solamente ejecutamos el siguiente webdriver igualando options a options y ya nos mantendrá el driver sin las cookies.

In [5]:
driver = webdriver.Chrome(options=options)    # abre una ventana de chrome 

In [6]:
df_event = pd.read_csv('../data/ENTRADAS.csv')

In [9]:
df_event = df_event.rename(columns={'ubicacion': 'local'})

In [10]:
df_event.head()

,nombre,local,precio,enlace,año,mes,dia,hora
0,El Rey León,Teatro Lope de Vega,49.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00:00
1,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00:00
2,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00:00
3,El Rey León,Teatro Lope de Vega,67.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00:00
4,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00:00


In [11]:
len(df_event["local"].unique())

41

In [13]:
df_event["local"].unique()

array(['Teatro Lope de Vega', 'Bajo la Gran Carpa', 'Nuevo Teatro Alcalá',
       'Teatro Rialto', 'WiZink Center', 'Sala La Riviera',
       'Sala La Paqui', 'Palacio Municipal de IFEMA MADRID',
       'The World Of Hans Zimmer: A New Dimension', 'Teatro La Latina',
       'Sala Siroco', 'Teatro Amaya', 'Teatro Arlequín Gran Vía',
       'Parque Enrique Tierno Galván', 'Teatro Maravillas',
       'Aeródromo de Cuatro Vientos', 'Teatro Lara - Sala Cándido Lara',
       'Teatro Muñoz Seca', 'Teatro EDP Gran Vía',
       'Recinto Ferial de IFEMA - Madrid', 'Pica Pica - Bailando',
       'Teatro Diéguez', 'Teatro Coliseum', 'Sala El Sol', 'Sala Capitol',
       'Teatro Reina Victoria', 'OBK', 'Gruta 77',
       'Gran Teatro CaixaBank Príncipe Pío', 'Cívitas Metropolitano',
       'Sala Villanos', 'Teatro Bellas Artes', 'Teatro Alcázar',
       'Recinto Ferial de IFEMA Madrid', 'Umusic Hotel Teatro Albéniz',
       'Teatro Infanta Isabel', 'PAX Madrid',
       'The Music Station Príncipe P

#### 1. Empezamos el scrapeo

- Creo una función con la que se me genere una lista, con los nombres de los locales y la ubicación de estos. 
- Esta ubicación se busca a través del scrapeo en google de la ubicación de los locales que le digo.

In [14]:
def obtener_ubicacion_local(local):
    ubicacion_dict = {}

    try:
        # Realizar búsqueda en Google y obtener el primer resultado
        query = f"ubicación {local}"
        url = f"https://www.google.com/search?q={query}"

        # Abrir la página en el navegador
        driver.get(url)

        # Esperar un breve momento para que la página cargue (puedes ajustar esto según sea necesario)
        time.sleep(3)

        # Obtener el contenido de la página después de que ha cargado
        page_content = driver.page_source

        # Analizar el contenido con Beautiful Soup
        soup = BeautifulSoup(page_content, 'html.parser')

        # Extraer la ubicación si está presente
        ubicacion_element = soup.find('div', class_='sXLaOe')
        ubicacion = ubicacion_element.get_text(strip=True) if ubicacion_element else None

        # Almacenar en el diccionario
        ubicacion_dict[local] = ubicacion

    except:
        print(f"La página para {local} no ha funcionado correctamente.")
        

    return ubicacion_dict


In [15]:
for local in df_event["local"].unique():
    resultado = obtener_ubicacion_local(local)
    if local in resultado:
        print(f"{local}, {resultado[local]}")
    else:
        print(f"{local}, No se encontró ubicación")

Teatro Lope de Vega, C.Gran Vía, 57, 28013 Madrid
Bajo la Gran Carpa, None
Nuevo Teatro Alcalá, C. de Jorge Juan, 62, 28009 Madrid
Teatro Rialto, C.Gran Vía, 54, 28013 Madrid
WiZink Center, Av. de Felipe II, s/n, 28009 Madrid
Sala La Riviera, P.º Bajo de la Virgen del Puerto, S/N, 28005 Madrid
Sala La Paqui, C. de Barceló, 11, Local B, 28004 Madrid
Palacio Municipal de IFEMA MADRID, Av. de la Capital de España, 5, 28042 Madrid
The World Of Hans Zimmer: A New Dimension, None
Teatro La Latina, Pl. de la Cebada, 2, 28005 Madrid
Sala Siroco, C. de San Dimas, 3, 28015 Madrid
Teatro Amaya, P.º del Gral. Martínez Campos, 9, 28010 Madrid
Teatro Arlequín Gran Vía, C. de San Bernardo, 5, 28013 Madrid
Parque Enrique Tierno Galván, C. Meneses, 4, 28045 Madrid
Teatro Maravillas, C. de Manuela Malasaña, 6, 28004 Madrid
Aeródromo de Cuatro Vientos, None
Teatro Lara - Sala Cándido Lara, Corre. Baja de San Pablo, 15, 28004 Madrid
Teatro Muñoz Seca, Pl. del Carmen, 1, 28013 Madrid
Teatro EDP Gran Vía, C

In [22]:
# Crear un diccionario para almacenar las ubicaciones con sus respectivos locales.

ubicaciones = {}  

for local in df_event["local"].unique():
    resultado = obtener_ubicacion_local(local)
    if local in resultado:
        ubicaciones[local] = resultado[local]
    else:
        ubicaciones[local] = "No se encontró ubicación"


In [24]:
ubicaciones

{'Teatro Lope de Vega': 'C.Gran Vía, 57, 28013 Madrid',
 'Bajo la Gran Carpa': None,
 'Nuevo Teatro Alcalá': 'C. de Jorge Juan, 62, 28009 Madrid',
 'Teatro Rialto': 'C.Gran Vía, 54, 28013 Madrid',
 'WiZink Center': 'Av. de Felipe II, s/n, 28009 Madrid',
 'Sala La Riviera': 'P.º Bajo de la Virgen del Puerto, S/N, 28005 Madrid',
 'Sala La Paqui': 'C. de Barceló, 11, Local B, 28004 Madrid',
 'Palacio Municipal de IFEMA MADRID': 'Av. de la Capital de España, 5, 28042 Madrid',
 'The World Of Hans Zimmer: A New Dimension': None,
 'Teatro La Latina': 'Pl. de la Cebada, 2, 28005 Madrid',
 'Sala Siroco': 'C. de San Dimas, 3, 28015 Madrid',
 'Teatro Amaya': 'P.º del Gral. Martínez Campos, 9, 28010 Madrid',
 'Teatro Arlequín Gran Vía': 'C. de San Bernardo, 5, 28013 Madrid',
 'Parque Enrique Tierno Galván': 'C. Meneses, 4, 28045 Madrid',
 'Teatro Maravillas': 'C. de Manuela Malasaña, 6, 28004 Madrid',
 'Aeródromo de Cuatro Vientos': None,
 'Teatro Lara - Sala Cándido Lara': 'Corre. Baja de San Pab

In [25]:
df_locales = pd.DataFrame(list(ubicaciones.items()), columns=['local', 'ubicacion'])

In [26]:
df_locales['local'] = df_locales['local'].str.rstrip(',')

In [27]:
df_locales.head()

,local,ubicacion
0,Teatro Lope de Vega,"C.Gran Vía, 57, 28013 Madrid"
1,Bajo la Gran Carpa,None
2,Nuevo Teatro Alcalá,"C. de Jorge Juan, 62, 28009 Madrid"
3,Teatro Rialto,"C.Gran Vía, 54, 28013 Madrid"
4,WiZink Center,"Av. de Felipe II, s/n, 28009 Madrid"


In [28]:
len(df_locales)

41

In [34]:
# Cargo en un .csv los locales con su respectiva ubicación. 

df_locales.to_csv('../data/df_locales.csv', index=False, encoding='utf-8')